#### Trying this to see how Pandas works 
https://mygeoblog.com/2017/01/13/your-gee-data-in-pandas/
https://mygeoblog.com/2017/10/06/from-gee-to-numpy-to-geotiff/

In [19]:
%matplotlib inline

Then we import in other Python packages we need.

In [20]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from ogr import gdal
from osgeo import osr, gdal
import time

# import datetime
# from matplotlib import dates
# import matplotlib.dates as mdates
# from pylab import *

In [21]:
ee.Initialize()

 #### import the EE asset (CHIRPS)

next we define datasets:

In [22]:
CHIRPS = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD') 
# info = CHIRPS.getInfo()
# print(info)

Define time range:

In [30]:
startYear = 2000
endYear = 2005

create list for years

In [31]:
years = range(startYear, endYear)

make a list with months

In [32]:
months = range(1,13)
print(months)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


Set date in ee date format

In [33]:
startDate = ee.Date.fromYMD(startYear,1,1)
endDate = ee.Date.fromYMD(endYear+1,12,31)

print(startDate)
## or alternatively using 
# startTime = datetime.datetime(1999, 1, 1)
# endTime = datetime.datetime(2016, 12, 31)

ee.Date({
  "type": "Invocation", 
  "arguments": {
    "year": 2000, 
    "day": 1, 
    "month": 1
  }, 
  "functionName": "Date.fromYMD"
})


Filter CHIRPS by date

In [34]:
filtCHIRPS = CHIRPS.filterDate(startDate, endDate).sort('system:time_start', False).select("precipitation")
# precip = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filterDate(startTime, endTime).select('precipitation')
# print(precip.first()) #get metadata from first element from Image Collection

In [35]:
# Define geographic domain
area = ee.Geometry.Rectangle(-20.0, 20.0, 20, 20.0)
areaInfo = area.getInfo()

lewa = ee.FeatureCollection('ft:1yrKHIrC4bnbgAP3l_ZCSm_0B_BCrI_hCm6p-RXnc')
lewaInfo = lewa.getInfo()

In [36]:
# calculate the monthly mean
def calcMonthlyMean(imageCollection):
    mylist = ee.List([])
    for y in years:
        for m in months:
            w = imageCollection.filter(ee.Filter.calendarRange(y, y, 'year')).filter(ee.Filter.calendarRange(m, m, 'month')).sum();
            mylist = mylist.add(w.set('year', y).set('month', m).set('date', ee.Date.fromYMD(y,m,1)).set('system:time_start',ee.Date.fromYMD(y,m,1)))
    return ee.ImageCollection.fromImages(mylist)

run the calcMonthlyMean function

In [37]:
monthlyCHIRPS = ee.ImageCollection(calcMonthlyMean(filtCHIRPS))

select the region of interest, 25000 is the cellsize in meters

In [48]:
monthlyCHIRPS = monthlyCHIRPS.getRegion(lewa, 500, "epsg:4326").getInfo()

AttributeError: 'list' object has no attribute 'getRegion'

Plot January (index=0)

In [39]:
January = pd.DataFrame(monthlyCHIRPS, columns = monthlyCHIRPS[0])

In [40]:
# remove first line 
January = January[1:]

In [41]:
# make sure unicode characters are removed
January['id'] = January['id'].str.decode('utf-8').replace(u'\xf1', 'n').astype('int')

In [42]:
# print(startTime)
print(January)

       id longitude  latitude                                         time  \
1       0   37.4216  0.132502   {u'type': u'Date', u'value': 946684800000}   
2       1   37.4216  0.132502   {u'type': u'Date', u'value': 949363200000}   
3       2   37.4216  0.132502   {u'type': u'Date', u'value': 951868800000}   
4       3   37.4216  0.132502   {u'type': u'Date', u'value': 954547200000}   
5       4   37.4216  0.132502   {u'type': u'Date', u'value': 957139200000}   
6       5   37.4216  0.132502   {u'type': u'Date', u'value': 959817600000}   
7       6   37.4216  0.132502   {u'type': u'Date', u'value': 962409600000}   
8       7   37.4216  0.132502   {u'type': u'Date', u'value': 965088000000}   
9       8   37.4216  0.132502   {u'type': u'Date', u'value': 967766400000}   
10      9   37.4216  0.132502   {u'type': u'Date', u'value': 970358400000}   
11     10   37.4216  0.132502   {u'type': u'Date', u'value': 973036800000}   
12     11   37.4216  0.132502   {u'type': u'Date', u'value': 975

In [43]:
# print(endTime)
# get Longitude
lons = np.array(January.longitude)
# get Latitude
lats = np.array(January.latitude)

In [44]:
# datetime.datetime?
data = np.array(January.precipitation)

Separate the information into column headers and data

In [45]:
# get the unique coordinates
uniqueLats = np.unique(lats)
uniqueLons = np.unique(lons)

In [46]:
# get number of columns and rows from coordinates
ncols = len(uniqueLons)    
nrows = len(uniqueLats)

In [47]:
# determine pixelsizes
ys = uniqueLats[1] - uniqueLats[0] 
xs = uniqueLons[1] - uniqueLons[0]

In [ ]:
point = {'type':'Point', 'coordinates':[ -116.88629,36.56122]};  # death valley (should be stable)
info = precip.getRegion(point,500).getInfo()
print(info)

In [ ]:
ee.ImageCollection.getRegion?
lewaRegionInfo = precip.getRegion(lewa, 1000, "EPSG:4326") #.getInfo()
print(lewaRegionInfo)

We separate the information returned into column headers and data.

In [ ]:
# extract the header column names
header = info[0]
# create a Numpy array of the data
data = array(info[1:])

Next we extract time information and convert it to at Python datatime data type.

Extract data columns that you want to display on the plot:

In [ ]:
iBands = [header.index(yBand)] #yBandList
yData = data[0:,iBands].astype(np.float)

And we use matplotlib to plot values

In [ ]:
# matplotlib date format object
fig = figure(figsize=(12,8), dpi=80)

# plot the band values
ax1 = fig.add_subplot(211)
ax1.plot(time, yData[:], '+', color="red", label="Death Valley")
# ax1.plot(time, yData[:,3], 'o', color="magenta",  label="Band 4")
ax1.legend(loc='best')
ax1.grid(True)

plt.title('Precip values as a function of time')
ax1.set_ylabel('Band Values')

### Now initiate precip values for Lewa

In [ ]:
# deathvalley = {'type':'Point', 'coordinates':[ -116.88629,36.56122]};  # death valley (should be stable)
# dv_info = precip.getRegion(deathvalley,500).getInfo()

In [ ]:
lewa = {'type':'Point', 'coordinates':[ 37.4, 0.2]};  # lewa valley (should be stable)
lewa_info = precip.getRegion(lewa,500).getInfo()

In [ ]:
# lewa = ee.Geometry.Rectangle(33, -0.2, 34, 0.2)
# lewa_info = precip.getRegion(lewa,500).getInfo()

We separate the information returned into column headers and data.

In [ ]:
# extract the header column names
header = lewa_info[0]
# create a Numpy array of the data
data = array(lewa_info[1:])

Next we extract time information and convert it to at Python datatime data type.

In [ ]:
# extract the time information
iTime = header.index('time')
# convert to Python datetime objects
time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]
# print(time)[:10]

Print first 10 asset timestamps

In [ ]:
for i in time[:10]:
    print(i)

Extract data columns that you want to display on the plot:

In [ ]:
iBands = [header.index(yBand)] #yBandList
yData = data[0:,iBands].astype(np.float)

In [ ]:
for i in iBands:
    print i

And we use matplotlib to plot values

In [ ]:
# matplotlib date format object
fig = figure(figsize=(12,8), dpi=80)

# plot the band values
ax2 = fig.add_subplot(211, sharex=ax1)
ax2.plot(time, yData[:], 'x', color="blue", label="Lewa")
# ax1.plot(time, yData[:,3], 'o', color="magenta",  label="Band 4")
ax2.grid(True)
start, end = ax2.get_xlim()

plt.title('CHIRPS values for Lewa')
ax2.set_ylabel('mm / pentad')

In [ ]:
type(yData)
# type(ax2)
# y = yData[ax2].resample('MS').mean()

In [ ]:
yData.head

In [ ]:
# Convert the timestamp to a numpy array
t = np.array([i.toordinal() for i in time])
t

In [ ]:
A = array([ t, ones(len(t))]).transpose()
# print(A)
b = precip     # could be NDVI
# print(b)
x = linalg.lstsq(A,b)[0] # obtaining the parameters
# # x = linalg.lstsq(A,b)[0] # obtaining the parameters
x

In [ ]:
b_hat = A.dot(x)

In [ ]:
fig2 = figure(figsize=(12,4), dpi=80)
plot(time,b_hat.transpose(),'r-',t,b,'o')
fig2.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
fig2.autofmt_xdate()

Create sequence lists (years, months, etc) for aggregations

Each asset spans a pentad. Each of first 5 pentads in a month have 5 days. The last pentad contains all the days from the 26th to the end of the month.

#### Calculate P for each year

In [ ]:
## JS function
# var YearlyChirps =  ee.ImageCollection.fromImages(
#   years.map(function (y) {
#     var w = precip.filter(ee.Filter.calendarRange(y, y, 'year'))
#              .sum();
#     return w.set('year', y)
#              .set('system:time_start',ee.Date.fromYMD(y,1,1))
#              .set('date', ee.Date.fromYMD(y,1,1));
# }).flatten());

Calculate monthly filter

In [ ]:
months = ee.List.sequence(1,12)
m = ee.Filter.calendarRange(months, months, 'months')
# m = precip.filter(ee.Filter.calendarRange(months, months, 'months')).sum()
# print(months)
print(m)

Calculate annual filter

In [ ]:
years = ee.List.sequence(1999,2016)
y = ee.Filter.calendarRange(years, years, 'year')
# y = precip.filter(ee.Filter.calendarRange(years, years, 'year')).sum()
print(y)

#### List values of assets

In [ ]:
months = {'1':'Jan', '2':'Feb', '3':'Mar', '4':'Apr', '5':'May', '6':'June', '7':'July', '8':'Aug', '9':'Sept', '10':'Oct', '11':'Nov', '12':'Dec'}

In [ ]:
for m in enumerate(months):
    print m
    print m[1]

In [ ]:
## build function which builds monthly cumulative image stack
def build_monthly(p):
    'Jan' = asset[1:6]
    'Feb' = asset[1:6]
    precip = 0
    while p < 6:
        precip += precip
        

In [ ]:
w = precip.filter(y.sum(), m.sum())

In [ ]:
y = w.set('year', y).set('system:time_start',ee.Date.fromYMD(y,1,1)).set('date', ee.Date.fromYMD(y,1,1)).flatten()

In [ ]:
YearlyChirps =  ee.ImageCollection.fromImages(
  years.map(function (y) {

def cumulative_chirps(y): ## start here
    return img.reduceRegions(xxxx, ee.Reducer.mean().forEachBand(img,200))
            var w = precip.filter(ee.Filter.calendarRange(y, y, 'year'))
             .sum();
    return w.set('year', y)
             .set('system:time_start',ee.Date.fromYMD(y,1,1))
             .set('date', ee.Date.fromYMD(y,1,1));
}).flatten());

#### Calculate P for each month

In [ ]:
# var MonthlyChirps =  ee.ImageCollection.fromImages(
#   years.map(function (y) {
#   return months.map(function(m){
#     var w = precip.filter(ee.Filter.calendarRange(y, y, 'year'))
#              .filter(ee.Filter.calendarRange(m, m, 'month'))
#              .sum();
#     return w.set('year', y)
#              .set('month', m)
#              .set('system:time_start',ee.Date.fromYMD(y,m,1))
#              .set('date', ee.Date.fromYMD(y,m,1));
# });
# }).flatten());

In [ ]:
y = months.map(m)

In [ ]:
monthlyChirps = ee.ImageCollection.fromImages(
  years.map(y), months.map(m)
    {w = precip.filter(ee.Filter.calendarRange(y, y, 'year'))
             .filter(ee.Filter.calendarRange(m, m, 'month'))
             .sum();
    return w.set('year', y)
             .set('month', m)
             .set('system:time_start',ee.Date.fromYMD(y,m,1))
             .set('date', ee.Date.fromYMD(y,m,1));
})
}).flatten())